[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming Methods

## Optimization - Covering Scattered Data by Discs

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.001 | 11/05/2025 | Royi Avital | Added analysis of `K` for K-Means                                  |
| 1.0.000 | 09/05/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0092DeepLearningResNet.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.cluster import KMeans

# Deep Learning

# Image Processing & Computer Vision

# Optimization
import cvxpy as cp

# Miscellaneous
import math
import os
from platform import python_version
import random

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Color Palettes
lMatPltLibclr   = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] #<! Matplotlib default color palette
lFlexokiClr     = ['#D14D41', '#DA702C', '#D0A215', '#879A39', '#3AA99F', '#4385BE', '#8B7EC8', '#CE5D97'] #<! Flexoki (Obsidian) Main
lFlexokiSatClr  = ['#AF3029', '#BC5215', '#BC5215', '#66800B', '#24837B', '#205EA6', '#5E409D', '#A02F6F'] #<! Flexoki (Obsidian) Saturated
lFlexokiGrayClr = ['#100F0F', '#1C1B1B', '#282726', '#343331', '#403E3C', '#55524E', '#878580', '#CECDC3'] #<! Flexoki (Obsidian) Grayscale
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants


In [ ]:
# Courses Packages


In [ ]:
# General Auxiliary Functions


## Service for Scattered Customers

Let $\left\{ \boldsymbol{x}_{i} \right\}_{i = 1}^{N}$ be set of points, representing location in 2D.  
In our case, they may represent the location of customers over a region of service.

![](https://i.imgur.com/pBWoRT0.png)
<!-- ![](https://i.postimg.cc/vBJpNbTK/Diagrams-Scattered-Data-Covering-001.png) -->

A unit of service has $r$ as its operational radius.  
Namely, a unit of service can handle all customers within its reach.  

The objective is to find the minimal service units and their location to cover all customers.

![](https://i.imgur.com/kpCix3t.png)
<!-- ![](https://i.postimg.cc/QNnBVzPy/Diagrams-Scattered-Data-Covering-002.png) -->

* <font color='brown'>(**#**)</font> The above model simplifies the problem by assuming the capacity of a single unit of service is unlimited or at least higher than the number of customers within any $r$ radius.

In [ ]:
# Parameters

# Data
numSamples = 120
dataDim    = 2

# Radius Constraint
maxRadius = 0.5

# Visualization
tuAxisLim = (-1.6, 1.6)

In [ ]:
# Load / Generate Data

# Generate data as a mixture of distributions
mX = np.r_[np.random.uniform(low = -1, high = 1, size = (numSamples // 4, dataDim)), 
           np.random.normal(loc = 0, scale = 0.6, size = (numSamples // 4, dataDim)), 
           np.random.logistic(loc = 0, scale = 0.2, size = (numSamples // 4, dataDim)),
           np.random.laplace(loc = 0, scale = 0.4, size = (numSamples // 4, dataDim)),]

In [ ]:
# Plot the Data
hF, hA = plt.subplots(figsize = (6, 4))
hA.scatter(mX[:, 0], mX[:, 1], s = 40, c = lFlexokiClr[0], alpha = 0.75, label = 'Customer Location')
hA.set_title('Scattered Data')
hA.set_xlabel(r'$x_1$')
hA.set_ylabel(r'$x_2$')
hA.axis('equal')

hA.legend();

## Approach 001 - Disc Covering

The approach is based on the [Disk Covering Problem](https://en.wikipedia.org/wiki/Disk_covering_problem).  
Assuming the data can be enclosed by a circle, one can use the optimal solutions of the _Disc Covering Problem_.


### Structure of the Solution

1. Find the minimum area enclosing circle.
2. Scale the problem.
3. Find the optimal solution by [Erich Friedman - Circles Covering Circles](https://erich-friedman.github.io/packing/circovcir).

* <font color='brown'>(**#**)</font> For a fast approximation of the solution see: 
  - [A Fast 25/6 Approximation for the Minimum Unit Disk Cover Problem](https://arxiv.org/abs/1406.3838).
  - [Experiments with Unit Disk Cover Algorithms for Covering Massive Pointsets](https://arxiv.org/abs/2205.01716).

<!-- https://math.stackexchange.com/questions/1269601 -->

### Step 001 - Minimum Area Enclosing Circle

The minimum area enclosing circle is given by:

$$\begin{align*}
\arg \min_{r, \boldsymbol{c}} \quad & r   \\
\text{subject to} \quad & \begin{aligned} 
{\left\| \boldsymbol{c} - \boldsymbol{x}_{i} \right\|}_{2}^{2} & \leq r, \; \forall i \\
\end{aligned}
\end{align*}$$



In [ ]:
# Optimization Problem

vC          = cp.Variable(dataDim) #<! Center (x, y) of the circle
valRadius   = cp.Variable(1)       #<! Radius of the circle

cpObjFun = cp.Minimize(valRadius)                                             #<! Objective Function
cpConst  = [cp.norm(mX[ii, :] - vC) <= valRadius for ii in range(numSamples)] #<! Constraint per each sample
oCvxPrb  = cp.Problem(cpObjFun, cpConst)                                      #<! Problem definition

oCvxPrb.solve(solver = cp.CLARABEL) #<! Solve the problem

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

In [ ]:
# Decode

# Extract the values as NumPy arrays
vC        = vC.value
valRadius = valRadius.value

In [ ]:
# Display Result

hF, hA = plt.subplots(figsize = (16, 8))
hA.scatter(mX[:, 0], mX[:, 1], s = 50, label = 'Samples')
patchCirc = Circle(vC, valRadius, color = 'r', lw = 2.5, fill = False, label = 'Bounding Circle')
hA.add_patch(patchCirc)
hA.scatter(vC[0], vC[1], s = 50, label = 'Center of Circle')
hA.set_aspect('equal')
hA.set_xlim(tuAxisLim)
hA.set_ylim(tuAxisLim)
hA.set_xlabel(r'$x_1$')
hA.set_ylabel(r'$x_2$')
hA.set_title('Data Samples')

hA.legend();

## Approach 002 - Clustering

This approach uses [Clustering](https://en.wikipedia.org/wiki/Cluster_analysis) to partition data into clusters.  
The [K-Means Algorithm](https://en.wikipedia.org/wiki/K-means_clustering) is known to create clusters shaped as symmetric blobs:

$$ \arg \min_{ \left\{ \mathcal{X}_{k} \right\}_{k = 1}^{K} } \sum_{k = 1}^{K} \sum_{\boldsymbol{x}_{i} \in \mathcal{X}_{k}} {\left\| \boldsymbol{x}_{i} - \boldsymbol{\mu}_{k} \right\|}_{2}^{2} $$

Where:

 * $\mathcal{X}_{k}$ - The $k$ -th cluster. Namely all points assigned to the cluster.
 * $\boldsymbol{\mu}_{k}$ - The centroid (Average) of the $k$ -th cluster.

The above find the set of clusters which minimizes the sum of squared distance from each cluster's centroid.

The classic algorithm can not enforce maximum size of the blob hence can not be used directly to solve the problem.

* <font color='brown'>(**#**)</font> One could use the classic K-Means algorithm and increase the `K` parameter until the largest blob is smaller then the required radius.
* <font color='brown'>(**#**)</font> For formulation of the K-Means using Integer Programming see:
    - [Kolos Cs. Agoston, Marianna E. Nagy - Mixed Integer Linear Programming Formulation for K-Means Clustering Problem](https://link.springer.com/article/10.1007/s10100-023-00881-1).
    - [Yet Another Math Programming Consultant - Clustering Models](https://yetanothermathprogrammingconsultant.blogspot.com/2021/05/clustering-models.html).

In [ ]:
# Analysis of K-Means
lK = [2, 5, 10, 15]

hF, hA = plt.subplots(1, len(lK), figsize = (16, 4))
hA = hA.flat

for ii, K in enumerate(lK):
    # K-Means Clustering
    oKMean = KMeans(n_clusters = K)
    vL     = oKMean.fit_predict(mX) #<! Labels of the clusters
    mC     = oKMean.cluster_centers_ #<! Centers of the clusters``

    hA[ii].scatter(mX[:, 0], mX[:, 1], s = 50, c = vL, label = 'Samples')
    hA[ii].scatter(mC[:, 0], mC[:, 1], s = 200, c = lFlexokiClr[0], label = 'Cluster Center', marker = 'X')
    for kk in range(K):
        circLabel = 'Bounding Circle' if kk == 0 else '_nolegend_'
        patchCirc = Circle(mC[kk, :], maxRadius, color = 'r', lw = 2.5, fill = False, label = circLabel)
        hA[ii].add_patch(patchCirc)
    
    hA[ii].set_aspect('equal')
    hA[ii].set_xlim(tuAxisLim)
    hA[ii].set_ylim(tuAxisLim)
    hA[ii].set_xlabel(r'$x_1$')
    hA[ii].set_ylabel(r'$x_2$')
    hA[ii].set_title(f'K-Means with {K} Clusters')

* <font color='brown'>(**#**)</font> For `K` large enough, the constraint will hold. Yet the objective might be far from optimal.

### Radius Bounded K-Means

Let $\boldsymbol{D} \in \mathcal{S}_{N} = \left\{ \boldsymbol{A} \in \mathbb{R}^{N \times N} \mid \boldsymbol{A} = \boldsymbol{A}^{T} \right\}$ be the Distance Matrix:

$$ {D}_{i, j} = {\left\| \boldsymbol{x}_{i} - \boldsymbol{x}_{j} \right\|}_{2}^{2} $$

Then the K-Means problem can be formulated as

$$ \arg \min_{ \left\{ \mathcal{X}_{k} \right\}_{k = 1}^{K} } \sum_{k = 1}^{K} \frac{1}{ 2 \left| \mathcal{X}_{k} \right| } \sum_{i, j} \left( \boldsymbol{1}_{\mathcal{X}_{k}} \boldsymbol{1}_{\mathcal{X}_{k}}^{T} \right) \odot \boldsymbol{D} $$

Where

 * $\left| \mathcal{X}_{k} \right|$ - The number of elements in the set $\mathcal{X}_{k}$.
 * $\boldsymbol{1}_{\mathcal{X}_{k}}$ - A vector of length $N$ with $1$ in indices matching the indices of the samples in $\mathcal{X}_{k}$ and zeros elsewhere.

The above form allows bounding the maximum distance between samples within the same cluster:

$$\begin{align*}
\arg \min_{ \left\{ \mathcal{X}_{k} \right\}_{k = 1}^{K} } \quad & \sum_{k = 1}^{K} \frac{1}{ 2 \left| \mathcal{X}_{k} \right| } \sum_{i, j} \left( \boldsymbol{1}_{\mathcal{X}_{k}} \boldsymbol{1}_{\mathcal{X}_{k}}^{T} \right) \odot \boldsymbol{D} \\
\text{subject to} \quad & \begin{aligned} 
{D}_{i, j} & \leq 4 {r}^{2}, \; \forall i, j \in \mathcal{X}_{k} \\
\end{aligned}
\end{align*}$$

The above can be formed into [Mixed Integer Programming](https://en.wikipedia.org/wiki/Integer_programming) problem:

$$\begin{align*}
\arg \min_{ \left\{ \mathcal{X}_{k} \right\}_{k = 1}^{K} } \quad & \sum_{k = 1}^{K} \frac{1}{ 2 {n}_{k} } \sum_{i, j} \left( \boldsymbol{1}_{\mathcal{X}_{k}} \boldsymbol{1}_{\mathcal{X}_{k}}^{T} \right) \odot \boldsymbol{D} \\
\text{subject to} \quad & \begin{aligned} 
\max_{i, j} \left( \boldsymbol{1}_{\mathcal{X}_{k}} \boldsymbol{1}_{\mathcal{X}_{k}}^{T} \right) \odot \boldsymbol{D} & \leq 4 {r}^{2}, \; \forall k = 1, 2, \ldots, K \\
\boldsymbol{1}^{T} \boldsymbol{1}_{\mathcal{X}_{k}} & = {n}_{k} \\
\sum {n}_{k} & = N \\
\sum \boldsymbol{1}_{\mathcal{X}_{k}} & = \boldsymbol{1} \\
\end{aligned}
\end{align*}$$

Where

 * $\max_{i, j} \left( \boldsymbol{1}_{\mathcal{X}_{k}} \boldsymbol{1}_{\mathcal{X}_{k}}^{T} \right) \odot \boldsymbol{D} \leq 4 {r}^{2}, \; \forall k = 1, 2, \ldots, K$ - Forces the maximum distance of 2 samples within a cluster to be less than $2r$.
 * $\boldsymbol{1}^{T} \boldsymbol{1}_{\mathcal{X}_{k}} = {n}_{k}$ - The sum of samples in the $k$ -th cluster is ${n}_{k}$.
 * $\sum {n}_{k} = N$ - The number of samples assigned to clusters is $N$.
 * $\sum \boldsymbol{1}_{\mathcal{X}_{k}} = \boldsymbol{1}$ - Each sample is assigned to a single cluster.

The problem can be approximated by a Convex problem as represented in [No More Than 6ft Apart: Robust K-Means via Radius Upper Bounds](https://arxiv.org/abs/2203.02502).



Since the approach above require commercial solvers, we'll define a simpler model:

$$\begin{align*}
\arg \min_{ \boldsymbol{B} \in \mathbb{N}^{N \times N} } \quad & \sum_{i} {B}_{i, i} \\
\text{subject to} \quad & \begin{aligned} 
{B}_{i, j} & \in \left\{ 0, 1 \right\} \\
\sum_{j} {B}_{i, j} & = 1, \; i \in \left\{ 1, 2, \ldots, N \right\} \\
\max_{i, j} \boldsymbol{B} \odot \boldsymbol{D} & \leq {r}^{2} \\
\max_{i} {B}_{i, j} & \leq {B}_{j, j}, \; j \in \left\{ 1, 2, \ldots, N \right\} \\
\end{aligned}
\end{align*}$$

Where
 - The objective function $\sum_{i} {B}_{i, i}$ counts the number of clusters.  
   Hence the minimization search for the least number of clusters.
 - ${B}_{i, j} \in \left\{ 0, 1 \right\}$ - Sets the matrix $\boldsymbol{B}$ to be a _Boolean_ indicator. ${B}_{i, j} = 1$ means the $i$ -th sample is assigned to the cluster defined by the $j$ -th sample.
 - $\sum_{j} {B}_{i, j} = 1, \; i \in \left\{ 1, 2, \ldots, N \right\}$ - Each element is assigned only once and only to a single cluster.
 - $\max_{i, j} \boldsymbol{B} \odot \boldsymbol{D} \leq {r}^{2}$ - The maximum distance of a sample from its centroid is bounded.
 - $\max_{i} {B}_{i, j} \leq {B}_{j, j}$ - If the $i$ -th sample is assigned to the cluster by the $j$ -th sample, then the $j$ -th sample is a center (Assigned to itself).

The objective minimizes the number of clusters.  
The problem is a [Integer Linear Programming](https://en.wikipedia.org/wiki/Integer_programming) (ILP) problem.

* <font color='brown'>(**#**)</font> The above model is K-Medoids which is similar to K-Means yet forces the center of the cluster to be an item within the cluster.

In [ ]:
def RadiusConstrainedKMeans( mX: np.ndarray, valRadius: float ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Solves a K-Means (K-Medoid) like problem with a radius constraint.
    Input:
        - mX: The data array of shape (numSamples, dataDim)
        - valRadius: The radius of the circle.
        - maxNumClusters: The maximum number of clusters.
    Output:
        - mC: The center of the circle.
    """
    
    numSamples = mX.shape[0]
    dataDim    = mX.shape[1]
    # The Squared Euclidean Distance matrix
    mD = sp.spatial.distance.squareform(sp.spatial.distance.pdist(mX, metric = 'sqeuclidean'))

    # The value B_ij represents whether element i belongs to the cluster centered on element j
    mB = cp.Variable((numSamples, numSamples), boolean = True) #<! Indicator matrix

    lConst = [] #<! List of constraints
    lConst.append(cp.sum(mB, axis = 1) == 1) #<! Each sample belongs to one cluster
    lConst.append(cp.max(cp.multiply(mB, mD)) <= valRadius * valRadius) #<! Limit the distance of each sample to its cluster center (Centroid)
    for jj in range(numSamples):
        lConst.append(mB[:, jj] <= mB[jj, jj]) #<! Each sample belongs to one cluster, The center assigned to itself

    cpObjFun = cp.Minimize(cp.sum(cp.diag(mB))) #<! Objective Function (Minimize the number of clusters)
    oCvxPrb  = cp.Problem(cpObjFun, lConst)

    oCvxPrb.solve(solver = cp.HIGHS)

    if oCvxPrb.status != 'optimal':
        return None, None, None, None

    mB = mB.value #<! The value of the indicator matrix

    vSIdx       = np.flatnonzero(np.diag(mB) == np.max(np.diag(mB))) #<! The cluster centers index
    numClusters = len(vSIdx) #<! The number of clusters
    # Assign label per sample by its cluster center
    vL = np.argmax(mB, axis = 1) #<! The label of each sample
    # The centroid of the cluster as its mean
    mC = np.zeros((numClusters, dataDim)) #<! The cluster centers

    # Map cluster labels to {0, 1, 2, ...}
    dClusterIdxMap = {val: idx for idx, val in enumerate(vSIdx)}
    vL             = np.vectorize(dClusterIdxMap.get)(vL)
    # The cluster centers (Centroids - Mean of the samples assigned to the cluster)
    for ii in range(numClusters):
        mC[ii, :] = np.mean(mX[vL == ii, :], axis = 0) #<! The cluster center
    
    return mC, vL, vSIdx, mB

In [ ]:
# Run the Optimization
mC, vL, vSIdx, mB = RadiusConstrainedKMeans(mX, maxRadius)

In [ ]:
# Plot the Clustering

hF, hA = plt.subplots(figsize = (16, 8))
hA.scatter(mX[:, 0], mX[:, 1], s = 50, c = vL, label = 'Samples')
hA.scatter(mC[:, 0], mC[:, 1], s = 200, c = lFlexokiClr[0], label = 'Cluster Center', marker = 'X')
for ii in range(mC.shape[0]):
    circleLabe = 'Clsuter Radius' if ii == 0 else '_nolegend_'
    patchCirc = Circle(mC[ii, :], maxRadius, lw = 2.5, fill = False, label = circleLabe)
    hA.add_patch(patchCirc)
hA.set_aspect('equal')
hA.set_xlim(tuAxisLim)
hA.set_ylim(tuAxisLim)
hA.set_xlabel(r'$x_1$')
hA.set_ylabel(r'$x_2$')
hA.set_title('Assigned Units Services')
hA.legend();

* <font color='red'>(**?**)</font> In some cases assigned samples are outside the circle, explain.
* <font color='red'>(**?**)</font> Some circles contain samples assigned to other. Why? What can be done?

<!--
1. The radius is enforced relative to a sample in the cluster. Yet the centroid is the mean of the assigned samples.
2. The radius is enforced for worse case. In real world one could adapt the radius of service to the farthest client.
-->